In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report, mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
import xgboost as xgb
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set random seed for reproducibility
np.random.seed(42)

# Create directories if they don't exist
import os
os.makedirs('models', exist_ok=True)
os.makedirs('reports', exist_ok=True)

In [3]:
# Load cleaned data
customers = pd.read_csv('/content/customers_cleaned.csv')
sales = pd.read_csv('/content/sales_cleaned.csv')
marketing = pd.read_csv('/content/marketing_cleaned.csv')
products = pd.read_csv('/content/products_cleaned.csv')


In [4]:
# Merge datasets
# Merge sales with customers on customer_id
merged = sales.merge(customers, on='customer_id', how='left')

# Merge with products on product_id
merged = merged.merge(products, on='product_id', how='left')

# Merge with marketing on customer_id (assuming marketing is per customer)
merged = merged.merge(marketing, on='customer_id', how='left')

# Validate merges
print(f"Merged dataset shape: {merged.shape}")
print(f"Null values per column:\n{merged.isnull().sum()}")

Merged dataset shape: (106549, 34)
Null values per column:
order_id                              0
customer_id                           0
product_id                            0
category_english_x                    0
price                                 0
freight_value                         0
payment_type                          0
payment_value                         0
order_purchase_timestamp            353
order_delivered_customer_date      2331
total_price                           0
year                                353
month                               353
weekday                             353
customer_unique_id                    0
customer_zip_code_prefix              0
city                                  0
state                                 0
total_orders                          0
total_spent                           0
last_order                            0
days_since_last_order                 0
churn_flag                            0
category_english_y   

In [5]:

# Aggregate to customer level
# Calculate order value
merged['order_value'] = merged['price'] * merged['quantity']

# Group by customer_id
customer_agg = merged.groupby('customer_id').agg(
    total_spend=('order_value', 'sum'),
    avg_order_value=('order_value', 'mean'),
    frequency=('order_value', 'count'),  # Number of orders
    last_purchase_date=('date', 'max'),
    first_purchase_date=('date', 'min'),
    # Marketing aggregates (assuming spend and conversions are per channel; aggregate per customer)
    total_marketing_spend=('spend', 'sum'),
    total_conversions=('conversions', 'sum'),
    num_campaigns=('channel', 'count'),  # Number of campaigns received
    # Other customer-level fields (take first, as they are static)
    churn_flag=('churn_flag', 'first'),
    state=('state', 'first'),
    payment_type=('payment_type', 'first'),
    category_english=('category_english', lambda x: x.mode()[0] if not x.mode().empty else 'Unknown')  # Most common category
).reset_index()


KeyError: 'quantity'

In [6]:

# Marketing variables per channel (pivot for average spend and conversion rate per channel)
marketing_pivot = merged.pivot_table(
    index='customer_id',
    columns='channel',
    values=['spend', 'conversions'],
    aggfunc='mean',
    fill_value=0
).reset_index()
marketing_pivot.columns = [f"{col[0]}_{col[1]}" if col[1] else col[0] for col in marketing_pivot.columns]

# Merge back
customer_agg = customer_agg.merge(marketing_pivot, on='customer_id', how='left')

# Validate final dataset
print(f"Final aggregated dataset shape: {customer_agg.shape}")
print(f"Null values in aggregated data:\n{customer_agg.isnull().sum()}")

# Save as model_input.csv
customer_agg.to_csv('model_input.csv', index=False)
print("Saved model_input.csv")

NameError: name 'customer_agg' is not defined

feature engineering

In [ ]:
# Load model_input.csv
data = pd.read_csv('model_input.csv')

# Derive RFM features
# Assuming current date is the max date in the dataset for recency
current_date = pd.to_datetime(data['last_purchase_date']).max()
data['recency'] = (current_date - pd.to_datetime(data['last_purchase_date'])).dt.days
data['tenure'] = (current_date - pd.to_datetime(data['first_purchase_date'])).dt.days
data['month_of_last_purchase'] = pd.to_datetime(data['last_purchase_date']).dt.month

# Marketing engagement score (e.g., conversions / spend, capped at 1)
data['marketing_engagement_score'] = np.where(data['total_marketing_spend'] > 0,
                                             data['total_conversions'] / data['total_marketing_spend'], 0)
data['marketing_engagement_score'] = np.clip(data['marketing_engagement_score'], 0, 1)

# Encode categorical variables
le = LabelEncoder()
data['payment_type_encoded'] = le.fit_transform(data['payment_type'])
data['state_encoded'] = le.fit_transform(data['state'])
data['category_english_encoded'] = le.fit_transform(data['category_english'])

# For channels, they are already pivoted into separate columns (e.g., spend_email, conversions_social)

# Normalize numerical features
scaler = StandardScaler()
numerical_cols = ['total_spend', 'avg_order_value', 'frequency', 'recency', 'tenure', 'total_marketing_spend', 'total_conversions', 'num_campaigns', 'marketing_engagement_score']
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

# Drop unnecessary columns
data = data.drop(['last_purchase_date', 'first_purchase_date', 'payment_type', 'state', 'category_english'], axis=1)

# Save feature-engineered data
data.to_csv('model_input_featured.csv', index=False)
print("Saved model_input_featured.csv")

Target Definition and Label Creation

In [ ]:
# Load featured data
data = pd.read_csv('model_input_featured.csv')

# Target: Churn Prediction (churn_flag)
X = data.drop(['customer_id', 'churn_flag'], axis=1)
y = data['churn_flag']

# Check class imbalance
print(f"Class distribution: {y.value_counts()}")

# Balance using SMOTE if imbalanced
if y.value_counts()[0] / y.value_counts()[1] > 1.5:  # Arbitrary threshold
    smote = SMOTE(random_state=42)
    X, y = smote.fit_resample(X, y)
    print("Applied SMOTE for balancing.")

# Final X and y ready

Train–Test Split and Baseline Model

In [ ]:
# Train-test split (70/30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Baseline: Logistic Regression
baseline_model = LogisticRegression(random_state=42)
baseline_model.fit(X_train, y_train)
y_pred_baseline = baseline_model.predict(X_test)
y_pred_proba_baseline = baseline_model.predict_proba(X_test)[:, 1]

# Evaluate
accuracy = accuracy_score(y_test, y_pred_baseline)
precision = precision_score(y_test, y_pred_baseline)
recall = recall_score(y_test, y_pred_baseline)
f1 = f1_score(y_test, y_pred_baseline)
roc_auc = roc_auc_score(y_test, y_pred_proba_baseline)

print(f"Baseline Logistic Regression Results:")
print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, ROC-AUC: {roc_auc:.4f}")

# Document in reports
with open('reports/baseline_results.txt', 'w') as f:
    f.write(f"Accuracy: {accuracy:.4f}\nPrecision: {precision:.4f}\nRecall: {recall:.4f}\nF1: {f1:.4f}\nROC-AUC: {roc_auc:.4f}\n")

Advanced Models and Hyperparameter Tuning

In [ ]:
# Models to train
models = {
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'XGBoost': xgb.XGBClassifier(random_state=42, eval_metric='logloss')
}

# Hyperparameter grids
param_grids = {
    'Decision Tree': {'max_depth': [5, 10, 15], 'min_samples_split': [2, 5, 10]},
    'Random Forest': {'n_estimators': [100, 200], 'max_depth': [10, 20], 'min_samples_split': [2, 5]},
    'XGBoost': {'n_estimators': [100, 200], 'max_depth': [3, 6], 'learning_rate': [0.1, 0.2]}
}

best_models = {}
results = {}

for name, model in models.items():
    grid = GridSearchCV(model, param_grids[name], cv=3, scoring='roc_auc', n_jobs=-1)
    grid.fit(X_train, y_train)
    best_models[name] = grid.best_estimator_
    y_pred = grid.predict(X_test)
    y_pred_proba = grid.predict_proba(X_test)[:, 1]
    results[name] = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'roc_auc': roc_auc_score(y_test, y_pred_proba)
    }
    print(f"{name} Best Params: {grid.best_params_}")
    print(f"{name} Results: {results[name]}")

# Compare with baseline
results_df = pd.DataFrame(results).T
results_df.to_csv('reports/model_comparison.csv')
print("Saved model_comparison.csv")

# Select best model (highest ROC-AUC)
best_model_name = max(results, key=lambda x: results[x]['roc_auc'])
best_model = best_models[best_model_name]
print(f"Selected Best Model: {best_model_name}")

Model Interpretation and Insights

In [ ]:
# Feature importance (for tree-based models)
if hasattr(best_model, 'feature_importances_'):
    importances = best_model.feature_importances_
    feature_names = X.columns
    plt.figure(figsize=(10, 6))
    sns.barplot(x=importances, y=feature_names)
    plt.title(f'Feature Importances - {best_model_name}')
    plt.savefig('reports/feature_importance.png')
    plt.show()

# Insights
print("Top Predictive Features:")
top_features = sorted(zip(feature_names, importances), key=lambda x: x[1], reverse=True)[:5]
for feat, imp in top_features:
    print(f"{feat}: {imp:.4f}")

# Business Insights
print("\nBusiness Insights:")
print("- Likely-to-churn customers: High recency, low frequency, low total_spend, low marketing engagement.")
print("- Key drivers: Recency and total_spend negatively impact churn; marketing engagement positively reduces it.")

Model Evaluation on Test Data

In [ ]:
# Evaluate best model on test
y_pred_test = best_model.predict(X_test)
y_pred_proba_test = best_model.predict_proba(X_test)[:, 1]

# Confusion matrix and report
cm = confusion_matrix(y_test, y_pred_test)
print("Confusion Matrix:\n", cm)
print("Classification Report:\n", classification_report(y_test, y_pred_test))

# Plot confusion matrix
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.savefig('reports/confusion_matrix.png')
plt.show()

# Compare train vs test (to check overfitting)
y_pred_train = best_model.predict(X_train)
train_auc = roc_auc_score(y_train, best_model.predict_proba(X_train)[:, 1])
test_auc = roc_auc_score(y_test, y_pred_proba_test)
print(f"Train ROC-AUC: {train_auc:.4f}, Test ROC-AUC: {test_auc:.4f}")
if train_auc - test_auc > 0.1:
    print("Potential overfitting detected.")

Model Preservation and Documentation

In [ ]:
# Save the best model
joblib.dump(best_model, f'models/{best_model_name.lower().replace(" ", "_")}_model.pkl')

# Save preprocessing (scaler, encoder)
joblib.dump(scaler, 'models/scaler.pkl')
joblib.dump(le, 'models/label_encoder.pkl')

# Document key insights
with open('reports/model_documentation.txt', 'w') as f:
    f.write(f"Best Model: {best_model_name}\n")
    f.write(f"Best Params: {best_models[best_model_name].get_params()}\n")
    f.write(f"Test Results: {results[best_model_name]}\n")
    f.write("Key Insights: See feature_importance.png and business interpretations above.\n")

print("Model and documentation saved.")